0. Import the libraries

In [2]:
import pandas as pd
import numpy as np
import re

1. Read the preprocessed-dataset

In [3]:
product_link = './preprocessed-data/product.pkl'

df = pd.read_pickle(product_link)

# drop if any column is nan
for field in df.columns:
    df = df[df[field].notna()]
    
# drop duplicate id
df.drop_duplicates(subset=['id'])

df.head()

,id,gender,masterCategory,subCategory,acticleType,baseColour,season,year,usage,productDisplayName,link
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,http://assets.myntassets.com/v1/images/style/p...
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,http://assets.myntassets.com/v1/images/style/p...
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,http://assets.myntassets.com/v1/images/style/p...
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,http://assets.myntassets.com/v1/images/style/p...
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,http://assets.myntassets.com/v1/images/style/p...


In [4]:
def changeQuoteSring(s):
    if "'" in s:
        return s.replace("'", "''")
    return s

changeQuoteSring("hello's")

"hello''s"

2. Manage the insert query

* Insert into category

In [7]:
main_categories = df['masterCategory'].unique()
main_categories

array(['Apparel', 'Accessories', 'Footwear', 'Personal Care',
       'Free Items', 'Sporting Goods', 'Home'], dtype=object)

* Insert into subcategory

In [8]:
# Create grouping between master and sub category
category_sub = df.groupby(['masterCategory','subCategory']).size().reset_index()
category_sub.loc[category_sub['subCategory'] == 'Bags']['masterCategory'].to_list()[0]

'Accessories'

In [9]:
sub_categories = df['subCategory'].unique()
sub_categories

array(['Topwear', 'Bottomwear', 'Watches', 'Socks', 'Shoes', 'Belts',
       'Flip Flops', 'Bags', 'Innerwear', 'Sandal', 'Shoe Accessories',
       'Fragrance', 'Jewellery', 'Lips', 'Saree', 'Eyewear', 'Scarves',
       'Dress', 'Loungewear and Nightwear', 'Wallets', 'Apparel Set',
       'Headwear', 'Mufflers', 'Skin Care', 'Makeup', 'Free Gifts',
       'Ties', 'Accessories', 'Nails', 'Beauty Accessories',
       'Water Bottle', 'Skin', 'Eyes', 'Bath and Body', 'Gloves',
       'Sports Accessories', 'Cufflinks', 'Sports Equipment', 'Stoles',
       'Hair', 'Perfumes', 'Home Furnishing', 'Umbrellas', 'Wristbands',
       'Vouchers'], dtype=object)

* Insert into product

In [11]:
df.head(1)

,id,gender,masterCategory,subCategory,acticleType,baseColour,season,year,usage,productDisplayName,link
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,http://assets.myntassets.com/v1/images/style/p...


In [15]:
product_sql = './queries-code/cassandra/product.cql'

with open(product_sql, 'a') as f:
    for index, product in df.iterrows():
        f.write(f"INSERT INTO ecommerce.product (product_id, product_name, product_gender, product_quantity, product_color, product_type, product_price, image_link, subcategory, category) VALUES ({product.id}, '{changeQuoteSring(product.productDisplayName)}', '{product.gender}', 0, '{product.baseColour}', '{product.acticleType}', 0.0, '{product.link}', '{product.subCategory}', '{product.masterCategory}');\n")


- Insert into product_by_category

In [16]:
product_by_category_sql = './queries-code/cassandra/product_by_category.cql'

with open(product_by_category_sql, 'a') as f:
    for index, product in df.iterrows():
        f.write(f"INSERT INTO ecommerce.product_by_category (product_id, product_name, product_gender, product_quantity, product_color, product_type, product_price, image_link, category) VALUES ({product.id}, '{changeQuoteSring(product.productDisplayName)}', '{product.gender}', 0, '{product.baseColour}', '{product.acticleType}', 0.0, '{product.link}', '{product.masterCategory}');\n")

- Insert into product_by_subcategory

In [18]:
product_by_subcategory_sql = './queries-code/cassandra/product_by_subcategory.cql'

with open(product_by_subcategory_sql, 'a') as f:
    for index, product in df.iterrows():
        f.write(f"INSERT INTO ecommerce.product_by_subcategory (product_id, product_name, product_gender, product_quantity, product_color, product_type, product_price, image_link, subcategory) VALUES ({product.id}, '{changeQuoteSring(product.productDisplayName)}', '{product.gender}', 0, '{product.baseColour}', '{product.acticleType}', 0.0, '{product.link}', '{product.subCategory}');\n")

- Insert into product_by_gender

In [19]:
product_by_gender_sql = './queries-code/cassandra/product_by_gender.cql'

with open(product_by_gender_sql, 'a') as f:
    for index, product in df.iterrows():
        f.write(f"INSERT INTO ecommerce.product_by_gender (product_id, product_name, product_gender, product_quantity, product_color, product_type, product_price, image_link) VALUES ({product.id}, '{changeQuoteSring(product.productDisplayName)}', '{product.gender}', 0, '{product.baseColour}', '{product.acticleType}', 0.0, '{product.link}');\n")